In [8]:
from ple.games.flappybird import FlappyBird
from ple import PLE
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import RMSprop, sgd
from keras.layers.normalization import BatchNormalization
import random
from IPython.display import clear_output
from datetime import datetime
import os
import glob
import shutil
import time

In [13]:
start_time = time.time() #Utilisé pour regarder le temps écoulé lors de l'apprentissage


# Définition des paramètres, modifiables
epochs = 5000 #Nombre de parties à jouer
gamma = 0.8 # discount factor 0.99
alpha=0.6 # learning rate 0.1
prob = 0.8 #proba de jouer None lors d'un mouvement aléatoire
experience_replay = False # Activer l'experience replay ? sûrement nécessaire pour meilleur apprentissage
batchSize = 32 # Taille du mini batch
buffer = 5000 #Taille du buffer
print_delay = 100

# FAIRE ATTENTION AUX RECOMPENSES : Jouer peut être sur des récompenses à posteriori plutôt que ce dict
reward_dict = { "positive": 1, "negative": 0.0, "tick": 0, "loss": -1000.0, "win": 0.0} #values


# Définition des fonctions utilisées et initialisation des variables

def getstate(jeu): #On ne prend que les éléments 0,1,2,3 : vitesse, pos verticale et horizontale de l'oiseau
                                                          #et pos verticale du haut du prochain tuyau
    return list(jeu.getGameState().values())[0:4]

actions = [None, 119]
update=0
h=0 #Taille du vecteur buffer, initialisée à 0
replay = [] # vecteur de buffer initialisé vide
epsilon = 0 # Initialisation d'epsilon (exploration/exploitation)
score = 0 # Variable utilisée pour mémoriser le meilleur score obtenu sur les epochs parties
time_for_save = datetime.now().strftime('%d-%m-%Y-%H:%M:%S') # Pour l'enregistrement
path = os.path.join('log-files', time_for_save)
os.makedirs(path) # Création du dossier
filenames = glob.glob('*.ipynb') # Sauvegarde du fichier notebook utilisé pour générer le réseau
for filename in filenames:
    shutil.copy(filename, path)
    
# Variables utilisées pour voir l'évolution
scores = []
average = []
bests = []
games = []

#Initilisation du jeu "jeu" et du joueur "p"
jeu = FlappyBird()

#frame_skip = The number of times we skip getting observations while repeat an action
#num_steps = The number of times we repeat an action.
#force_fps = True pour ne pas jouer en temps réel (plus vite)
#display_screen = True pour afficher le déroulement de la partie en direct
#reward_values = This contains the rewards we wish to set give our agent

p = PLE(jeu, fps=30, frame_skip=1, num_steps=1, force_fps=True, display_screen=True, reward_values = reward_dict)

p.reset_game()


# Création du réseau de neurones
model = Sequential()
# # Première couche, 150 avec 8 inputs, les 8 composantes du vecteur d'état
# model.add(Dense(150, kernel_initializer='lecun_uniform', input_shape=(8,)))
# Première couche, 150 avec 4 inputs, les 4 composantes du vecteur d'état qu'on a choisis
model.add(Dense(150, kernel_initializer='lecun_uniform', input_shape=(4,)))
model.add(Activation('relu'))
model.add(BatchNormalization(axis=1))
# # Couche cachée, taille 300
# model.add(Dense(300, kernel_initializer='lecun_uniform'))
# model.add(Activation('relu'))
# Couche cachée, taille 150
model.add(Dense(150, kernel_initializer='lecun_uniform'))
model.add(Activation('relu'))
# Couche de sortie, taille 2 comme les 2 actions
model.add(Dense(2, kernel_initializer='lecun_uniform'))
model.add(Activation('linear'))
# Compilation du modèle
model.compile(loss='mse', optimizer="rmsprop")



#Début des parties, on en joue un nombre : epochs
for i in range(epochs):
    #Réinitialisation du jeu
    p.reset_game()
    #Acquisition de l'état du jeu (s)
    state = getstate(jeu)
    
    #Début de la boucle, tant que la partie n'est pas finie :
    while(not jeu.game_over()):
        
        # Evaluation de la valeur de la policy pour les deux actions : Q(s,a)
        qval = model.predict(np.array(state).reshape(1,len(state)), batch_size=batchSize)
 
        #Stratégie Exploration/exploitation 
        if (np.random.random() < epsilon): # On choisit une action au hasard, proba prob pour l'action None
            if np.random.random() < prob:
                action = actions[0]
            else:
                action = actions[1]
        else: # On choisit la meilleure action, qui maximise qval (Q(s,a))
            if qval[0][0] > qval[0][1]:
                action = actions[0]
            else:
                action = actions[1]

        #On fait l'action "action" (a), on observe le nouvel état new_state (s') et la récompense reward (r)
        r = p.act(action)
        
        #On observe la récompense liée à cette action
        if r == 0: #Dans ce cas l'oiseau est encore en vie, on donne une récompense de 1
            reward = 1
        elif r == 1: #L'oiseau a franchi un tuyau, on donne une récompense de  100
            reward = 100
        else: #L'oiseau a touché un tuyau, le sol ou le plafond, on fait reward = r = -1000 (cf reward_dict)
            reward = r
            
        new_state = getstate(jeu)   
        
        #Actualisation du score si meilleur que le précédent
        if jeu.getScore() > score :
            score = jeu.getScore()
            
        if not experience_replay:# si on n'a pas activé la mémoire
            #Calcul des valeurs Q(s',a) POURQUOI BATCH SIZE ????
            newQ = model.predict(np.array(new_state).reshape(1,len(state)), batch_size=batchSize)
            # Recherche du max
            maxQ = np.max(newQ)
            #Initialisation de y à Q(s,a)
            y = np.zeros((1,2))
            y[:] = qval[:]
            
            #Calcul de l'update
            if reward != reward_dict["loss"]: # Partie pas perdue
                update = (reward + gamma * maxQ)
            else:
                update = reward
                
            # On met la valeur "update" dans la composante de y qui correspond à l'action jouée
            if action == None:
                y[0][0] = update #target output
            else :
                y[0][1] = update
                
            #On entraine le réseau de neurones avec state en entrée, y en sortie
            model.fit(np.array(state).reshape(1, len(state)), y, batch_size=batchSize, epochs=1, verbose=0)
            
            #On actualise l'état
            state = new_state
            
            #clear_output(wait=True)
            
        else: #Si on a activé la mémoire
            
            if (len(replay) < buffer): #Si le buffer n'est pas plein, on ajoute le dernier sars
                replay.append((state, action, reward, new_state))
            else: #Si il est plein, on remplace une vieille valeur et fait de l'apprentissage
                
                if (h < (buffer-1)): #Si h est plus petit que la taille du vecteur replay :
                    h += 1 #On incrémente h pour changer la composante suivante du vecteur replay
                else: #Sinon (donc si h pointe vers le dernier élément du replay)
                    h = 0 # On remet h à 0
                # On remplace l'élément h par le sars nouvellement acquis
                replay[h] = (state, action, reward, new_state)
                
                #On prends batchSize éléments aléatoirement dans notre replay memory :
                minibatch = random.sample(replay, batchSize)
                
                # Réinitialisation de X_train et y_train
                X_train = []
                y_train = []
                
                #On prend les éléments dans minibatch, le but est de remplir X_train et y_train
                for memory in minibatch: #On parcourt les éléments sélectionnés
                    #On cherche le max de Q(s',a) :
                    #On stocke s,a,r,s pour chaque élément
                    old_state, action, reward, new_state = memory
                    #On calcule la q_value pour l'ancien état s Q(s,.)
                    old_qval = model.predict(np.array(old_state).reshape(1,len(old_state)), batch_size=1)
                    #On calcule la q_value pour le nouvel état s' Q(s',.)
                    newQ = model.predict(np.array(new_state).reshape(1,len(new_state)), batch_size=1)
                    #On stocke la valeur maximale de la q_value du nouvel état max(a)Q(s',a)
                    maxQ = np.max(newQ)
                    # Initialisation de y aux valeurs de Q(s,.) de l'ancien état
                    y = np.zeros((1,2))
                    y[:] = old_qval[:]
                    
                    #Calcul de l'update en fonction de si l'état est terminal ou non
                    if reward != reward_dict["loss"]: #Etat non terminal
                        update = (reward + (gamma * maxQ))
                    else: #Etat terminal
                        update = reward
                    # On met la valeur "update" dans la composante de y qui correspond à l'action jouée
                    if action == None:
                        y[0][0] = update
                    else :
                        y[0][1] = update
                        
                    #On ajoute old_state et y à X_train et y_train
                    X_train.append(np.array(old_state).reshape(len(old_state),))
                    y_train.append(np.array(y).reshape(2,))
                    
                #On met X_train et y_train sous forme de tableau
                X_train = np.array(X_train)
                y_train = np.array(y_train)
                
                #Entrainement du réseau avec batchSize éléments
                model.fit(X_train, y_train, batch_size=batchSize, epochs=1, verbose=0)
                
                #Mise à jour de l'état avec le nouvel état
                state = new_state
            #clear_output(wait=True)
    
    #Après la fin de la partie, on ajoute le score final à la liste des scores
    scores.append(jeu.getScore()-reward_dict["loss"])
    
    # Mise à jour de la stratégie exploitation / exploration
    if epsilon > 0.2:
        epsilon -= (1.0/epochs)
    
        
    #Sauvegarde du Q courant, et affichage de quelques donnée pour suivre l'apprentissage
    if ((i+1)%print_delay) == 0:
        print("")
        print("Game #: %s" % (i+1,))
        print("Meilleur score sur les ",print_delay," derniers essais =", np.max(scores))
        print("Score moyen sur les ",print_delay," derniers essais =", np.mean(scores))
        elapsed_time = time.time() - start_time
        printime = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))
        print("Temps écoulé : ", printime)
        average.append(np.mean(scores))
        bests.append(np.max(scores))
        games.append(i+1)
        print('Save done avec moyenne: ' + np.str(np.mean(scores)))
        scores = []
        now = datetime.now().strftime("%d-%m-%Y-%H:%M:%S")
        name = 'FlappyBird_'+now+'.dqf'
        dire = './log-files/'+time_for_save+'/'+name
        model.save(dire)
            
print("Best score =", score)
dire = './log-files/'+time_for_save+'/FlappyBird.dqf'
model.save(dire)


Game #: 100
Meilleur score sur les  100  derniers essais = 1.0
Score moyen sur les  100  derniers essais = 0.03
Temps écoulé :  00:00:18
Save done avec moyenne: 0.03

Game #: 200
Meilleur score sur les  100  derniers essais = 2.0
Score moyen sur les  100  derniers essais = 0.07
Temps écoulé :  00:00:38
Save done avec moyenne: 0.07

Game #: 300
Meilleur score sur les  100  derniers essais = 4.0
Score moyen sur les  100  derniers essais = 0.1
Temps écoulé :  00:01:02
Save done avec moyenne: 0.1

Game #: 400
Meilleur score sur les  100  derniers essais = 1.0
Score moyen sur les  100  derniers essais = 0.06
Temps écoulé :  00:01:20
Save done avec moyenne: 0.06

Game #: 500
Meilleur score sur les  100  derniers essais = 1.0
Score moyen sur les  100  derniers essais = 0.05
Temps écoulé :  00:01:39
Save done avec moyenne: 0.05

Game #: 600
Meilleur score sur les  100  derniers essais = 1.0
Score moyen sur les  100  derniers essais = 0.02
Temps écoulé :  00:01:59
Save done avec moyenne: 0.02


SystemExit: 

/home/ldeschamps/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Test du réseau de neurones

In [ ]:
from keras.models import load_model
import numpy as np
actions = [None, 119]
model = load_model('FlappyBird.dqf')

def FlappyPolicy(state, screen):
	state_list = list(state.values())[0:4]
	
	qval = model.predict(np.array(state_list).reshape(1,len(state_list)), batch_size=1)
	if qval[0][0] > qval[0][1]:
	    action = actions[0]
	else:
	    action = actions[1]
	print(action)
	return action


game = FlappyBird()
p = PLE(game, fps=30, frame_skip=1, num_steps=1, force_fps=True, display_screen=False)

p.init()
reward = 0.0

nb_games = 100
cumulated = np.zeros((nb_games))

for i in range(nb_games):
    p.reset_game()
    
    while(not p.game_over()):
        state = game.getGameState()
        screen = p.getScreenRGB()
        action=FlappyPolicy(state, screen) ### Your job is to define this function.
        
        reward = p.act(action)
        cumulated[i] = cumulated[i] + reward
    print('Game #', i, ' score :',cumulated[i]+5)

average_score = np.mean(cumulated+5)
max_score = np.max(cumulated+5)

print("Score moyen sur les 100 parties :", average_score)
print("Score max sur les 100 parties :", max_score)

if average_score > 15:
    print("YOU WOOOOOOOOOOOOOOOOOOON !!!!!")
else:
    print("Loser.")
        
